In [1]:
import numpy as np
import pandas as pd
import tabula

Read and OCR the 20th page

In [2]:
table = tabula.read_pdf("adsi-2015-full-report.pdf",pages=20)

Got stderr: Nov 27, 2020 12:45:38 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 27, 2020 12:45:38 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 27, 2020 12:45:38 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 27, 2020 12:45:41 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 27, 2020 12:45:41 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 27, 2020 12:45:42 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



Gain understanding of extra text read

In [3]:
table2 = table[0].copy()
table[0].head()

,Unnamed: 0,Unnamed: 1,Number of Accidental Deaths & Sui cides in India During 1967 to 2015,Unnamed: 2
0,Sl.,NaN,Accidental Deaths Suicides,NaN
1,No.,Year,Male Female Transgender Total Male Female Tran...,Total
2,(1),(2),(3) (4) (5) (6) (7) (8) (9),(10)
3,1.,1967,79131 47631 126762 22637 16192,38829
4,2.,1968,79159 47073 126232 24464 16224,40688


Pick 3rd Row for Column names

In [4]:
table2.columns = table2.iloc[2]

Drop the earlier columns

In [5]:
table2.drop([0,1,2],inplace=True)

Set column "(1)" as Dataframe Index

In [6]:
table2.set_index("(1)",inplace=True)

In [7]:
table2.tail(5)

2,(2),(3) (4) (5) (6) (7) (8) (9),(10)
(1),,,
45.,2011,302420 88464 390884 87839 47746,135585
46.,2012,306061 88921 394982 88453 46992,135445
47.,2013,312670 87847 400517 90543 44256,134799
48.,2014,354573 97078 106 451757 89129 42521 16,131666
49.,2015,328241 85169 47 413457 91528 42088 7,133623


Split the concatenated column "(3) (4) (5) (6) (7) (8) (9)" into its individual columns and conserve their column naming

In [8]:
table2[["(3)","(4)","(5)","(6)","(7)","(8)","(9)"]] = table2['(3) (4) (5) (6) (7) (8) (9)'].str.split(expand=True)\
                    .rename({0:"(3)",1:"(4)",2:"(5)",3:"(6)",4:"(7)",5:"(8)",6:"(9)"},axis=1)

Drop the concatenated column "(3) (4) (5) (6) (7) (8) (9)"

In [9]:
table2 = table2.drop('(3) (4) (5) (6) (7) (8) (9)',axis=1)

Re-order columns as per original PDF schema

In [10]:
col_order = ["(2)","(3)","(4)","(5)","(6)","(7)","(8)","(9)","(10)"]
table2 = table2[col_order]

In [11]:
table2.tail()

2,(2),(3),(4),(5),(6),(7),(8),(9),(10)
(1),,,,,,,,,
45.,2011,302420,88464,390884,87839,47746,None,None,135585
46.,2012,306061,88921,394982,88453,46992,None,None,135445
47.,2013,312670,87847,400517,90543,44256,None,None,134799
48.,2014,354573,97078,106,451757,89129,42521,16,131666
49.,2015,328241,85169,47,413457,91528,42088,7,133623


Notice that the values for the 6th, 7th and 8th Column are shifted to the left except for the last two rows.  
The 5th and 9th columns are missing values upto the year 2013.  
So basically, move the values of 5th, 6th and 7th to one column in the right.  
For all rows except the ones which are already filled with values.

In [12]:
table3 = table2.copy()
#Iterate throught the DataFrame row by row
for index, row in table3.iterrows():
    #Whenever the 9th Column has "None"
    if( str(row.loc["(9)"]) == "None" ):
        #Replace Missing values with NaN
        row.loc["(9)"] = np.nan
        #Move the rest of the columns to its right after typecasting as integer.
        row.loc["(8)"] = int(row.loc["(7)"])
        row.loc["(7)"] = int(row.loc["(6)"])
        row.loc["(6)"] = int(row.loc["(5)"])
        row.loc["(5)"] = np.nan

Rename the index according the PDF and typecast it to integer

In [13]:
table3.index = table3.index.set_names("Sl. No.").astype('float64').astype('int64')

In [15]:
table3.head()

2,(2),(3),(4),(5),(6),(7),(8),(9),(10)
Sl. No.,,,,,,,,,
1,1967,79131,47631,NaN,126762,22637,16192,NaN,38829
2,1968,79159,47073,NaN,126232,24464,16224,NaN,40688
3,1969,82105,48650,NaN,130755,25947,17686,NaN,43633
4,1970,89210,50542,NaN,139752,28846,19582,NaN,48428
5,1971,68344,37257,NaN,105601,26326,17349,NaN,43675


Create Schema of Column-names from Original PDF in multi-level fashion

In [16]:
columns=[("Year",""),
         ('Accidental Deaths','Male'),
         ('Accidental Deaths','Female'),
         ('Accidental Deaths','Transgender'),
         ('Accidental Deaths','Total'),
         ('Suicides','Male'),
         ('Suicides','Female'),
         ('Suicides','Transgender'),
         ('Suicides','Total'),
        ]
columns

[('Year', ''),
 ('Accidental Deaths', 'Male'),
 ('Accidental Deaths', 'Female'),
 ('Accidental Deaths', 'Transgender'),
 ('Accidental Deaths', 'Total'),
 ('Suicides', 'Male'),
 ('Suicides', 'Female'),
 ('Suicides', 'Transgender'),
 ('Suicides', 'Total')]

Apply column-schema

In [17]:
table3.columns = pd.MultiIndex.from_tuples(columns)
table3

Year Accidental Deaths                            Suicides         \
                           Male Female Transgender   Total     Male Female   
Sl. No.                                                                      
1        1967             79131  47631         NaN  126762    22637  16192   
2        1968             79159  47073         NaN  126232    24464  16224   
3        1969             82105  48650         NaN  130755    25947  17686   
4        1970             89210  50542         NaN  139752    28846  19582   
5        1971             68344  37257         NaN  105601    26326  17349   
6        1972             69838  36346         NaN  106184    26923  16678   
7        1973             69613  36741         NaN  130654    25231  15576   
8        1974             73580  37044         NaN  110624    27791  18217   
9        1975             74987  38029         NaN  113016    26074  16816   
10       1976             73098  38513         NaN  111611    24042  17373   
11       1977             76013  41325         NaN  117338    23453  16265   
12       1978             78352  40242         NaN  118594    24137  16070   
13       1979             72448  36539         NaN  108987    22980  15237   
14       1980             76293  40619         NaN  116912    24188  17475   
15       1981             82328  39893         NaN  122221    23864  16381   
16       1982             82977  43016         NaN  125993    26520  18212   
17       1983             85577  42999         NaN  128576    27260  19319   
18       1984             90636  43992         NaN  134628    29296  21275   
19       1985             93568  46089         NaN  139657    30460  22351   
20       1986             98480  48543         NaN  147023    31271  23086   
21       1987            103727  48587         NaN  152314    34292  24276   
22       1988            110235  53287         NaN  163522    37755  26515   
23       1989            114953  54113         NaN  169066    40212  28532   
24       1990            117932  56469         NaN  174401    43451  30460   
25       1991            129142  58861         NaN  188003    46324  32126   
26       1992            133977  60933         NaN  194910    47481  32668   
27       1993            130499  61858         NaN  192357    49851  34393   
28       1994            132241  58194         NaN  190435    52752  36443   
29       1995            157219  65268         NaN  222487    52357  36821   
30       1996            156106  63988         NaN  220094    51206  37035   
31       1997            164876  69027         NaN  233903    56281  39548   
32       1998            185520  72889         NaN  258409    61686  43027   
33       1999            193652  78266         NaN  271918    65488  45099   
34       2000            186324  69559         NaN  255883    66032  42561   
35       2001            196129  74890         NaN  271019    66314  42192   
36       2002            194850  65272         NaN  260122    69332  41085   
37       2003            197285  62340         NaN  259625    70221  40630   
38       2004            210190  67073         NaN  277263    72651  41046   
39       2005            224806  69369         NaN  294175    72916  40998   
40       2006            241210  73494         NaN  314704    75702  42410   
41       2007            262918  77876         NaN  340794    79295  43342   
42       2008            263809  78500         NaN  342309    80544  44473   
43       2009            276333  80688         NaN  357021    81471  45680   
44       2010            298262  86387         NaN  384649    87180  47419   
45       2011            302420  88464         NaN  390884    87839  47746   
46       2012            306061  88921         NaN  394982    88453  46992   
47       2013            312670  87847         NaN  400517    90543  44256   
48       2014            354573  97078         106  451757    89129  42521   
49       2015            328241

In [18]:
table3.to_csv("Outputs/CSV/Number_of_Accidental_Deaths_&_Suicides_in_India_During_1967_to_2015.csv")
table3.to_excel("Outputs/Excel/Number_of_Accidental_Deaths_&_Suicides_in_India_During_1967_to_2015.xlsx")